In [ ]:
from pandas import DataFrame
from tqdm import tqdm
import numpy as np
import pandas as pd
import cv2
from ultralytics import YOLO
import os
import json
import time
import matplotlib.pyplot as plt

In [ ]:
DIRECTORY = 'data/SoccerNetGS/gamestate-2024/train/'


def load_data(dir: str = None) -> list:
    data = list()
    print('Wczytywanie danych...')
    for folder_name in tqdm(os.listdir(DIRECTORY)):
        folder_full_path = os.path.join(DIRECTORY, folder_name)
        if os.path.isdir(folder_full_path):
            with open(os.path.join(DIRECTORY, folder_name, 'Labels-GameState.json')) as f:
                d = json.load(f)
                data.append(d['annotations'])
    return data


def parse_data_to_df(data: list) -> pd.DataFrame:
    start = time.time()
    print('Tworzenie DataFrame...')
    flat_dict = dict()
    for annotations in tqdm(data):
        data_image = filter(lambda x: x['supercategory'] == 'object' 
                                      and x['bbox_pitch_raw'] is not None, annotations)
        for img in data_image:
            for key in img:
                if isinstance(img[key], dict):
                    for key2 in img[key]:
                        if key + '_' + key2 not in flat_dict:
                            flat_dict[key + '_' + key2] = []
                        flat_dict[key + '_' + key2].append(img[key][key2])
                else:
                    if key not in flat_dict:
                        flat_dict[key] = []
                    flat_dict[key].append(img[key])
    df = pd.DataFrame(flat_dict, index=flat_dict['id'])
    df = df.drop(columns=['id'])
    return df


In [ ]:
def flatten_data(data: list[list[dict]]) -> list:
    flat_list = [y for x in data for y in x]
    return flat_list

In [ ]:
def get_pitch_data_df(data: list) -> pd.DataFrame:
    pitch_data_dict = {'id': [], 'image_id': [], 'line_type': [], 'x': [], 'y': []}
    data = filter(lambda x: x['supercategory'] == 'pitch', data)
    for anotation in tqdm(data):
        for line in anotation['lines']:
            for data_point in anotation['lines'][line]:
                pitch_data_dict['id'].append(int(anotation['id']))
                pitch_data_dict['image_id'].append(int(anotation['image_id']))
                pitch_data_dict['line_type'].append(line)
                pitch_data_dict['x'].append(data_point['x'])
                pitch_data_dict['y'].append(data_point['y'])
    pitch_df = pd.DataFrame(pitch_data_dict)
    return pitch_df

In [ ]:
def denormalize_coordinates(x: float, y: float, img_width: int, img_height: int) -> tuple[int, int]:
    return int(x * img_width), int(y * img_height)


def draw_points_on_image(image: np.ndarray, coords: pd.DataFrame) -> np.ndarray:
    image_height, image_width, _ = image.shape

    colors = {
        # 3 punkrtowce
        'Circle central': (0, 255, 255),
        'Circle left': (255, 255, 255),
        'Circle right': (140, 159, 191),
        # 'Goal left crossbar': (255, 128, 128),
        # 'Goal left post left': (128, 255, 128),
        # 'Goal left post right': (128, 128, 255),
        # 'Goal right crossbar': (255, 128, 255),
        # 'Goal right post left': (255, 255, 128),
        # 'Goal right post right': (128, 255, 255),
        # lewe pole karne
        'Big rect. left bottom': (0,0,0),
        'Big rect. left main': (0, 255, 0),
        'Big rect. left top': (255, 0, 0),
        'Small rect. left bottom': (64, 255, 128),
        'Small rect. left main': (192, 255, 32),
        'Small rect. left top': (32, 192, 255),
        #prawe pole karne        
        'Big rect. right bottom': (0, 0, 255),
        'Big rect. right main': (255, 255, 0),
        'Big rect. right top': (255, 0, 255),
        'Small rect. right bottom': (255, 32, 192),
        'Small rect. right main': (32, 255, 192),
        'Small rect. right top': (192, 32, 255),
        #linie
        'Middle line': (255, 64, 128),
        'Side line bottom': (128, 255, 64),
        'Side line left': (64, 128, 255),
        'Side line right': (128, 64, 255),
        'Side line top': (255, 128, 64)
    }
    for i, row in coords.iterrows():
        color = colors[row['line_type']] if row['line_type'] in colors.keys() else (0, 0, 0)
        x_pixel, y_pixel = denormalize_coordinates(row['x'], row['y'], image_width, image_height)
        cv2.circle(image, (x_pixel, y_pixel), radius=5, color=color, thickness=-1)
    # for category, points in coords.items():
    #     color = colors[category]
    #     for point in points:
    #         if category == 'Middle line':
    #             x_pixel, y_pixel = denormalize_coordinates(point['x'], point['y'], image_width, image_height)
    #             cv2.circle(image, (x_pixel, y_pixel), radius=5, color=color, thickness=-1)

    scale_factor = min(1920 / image_width, 1080 / image_height)
    if scale_factor < 1:
        image = cv2.resize(image, (int(image_width * scale_factor), int(image_height * scale_factor)))

    # cv2.imshow("Image with Points", image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return image

In [ ]:
line_to_yolo_class_mapping = {
    'Circle central': 0,
    'Circle left': 1,
    'Circle right': 2,
    # lewe pole karne
    'Big rect. left bottom': 3,
    'Big rect. left main': 4,
    'Big rect. left top': 5,
    'Small rect. left bottom': 6,
    'Small rect. left main': 7,
    'Small rect. left top': 8,
    #prawe pole karne        
    'Big rect. right bottom': 9,
    'Big rect. right main': 10,
    'Big rect. right top': 11,
    'Small rect. right bottom': 12,
    'Small rect. right main': 13,
    'Small rect. right top': 14,
    #linie
    'Middle line': 15,
    'Side line bottom': 16,
    'Side line left': 17,
    'Side line right': 18,
    'Side line top': 19
}

In [ ]:
def get_yolo_file_for_pitch(df: pd.DataFrame, split: str) -> None:
    file_contents = ['' for _ in range(df.image_id.nunique())]
    df = df[df['line_type'].isin(line_to_yolo_class_mapping.keys())]
    for i, row in tqdm(df.iterrows()):
        line_type: str = row['line_type']
        x: float = row['x']
        y: float = row['y']
        category: int = line_to_yolo_class_mapping[line_type]
        file_contents[int(str(row['image_id'])[-3:]) - 1] += f'{category} {x} {y}\n'
    for i, file in tqdm(enumerate(file_contents)):
        with open(f'datasets/football_lines/{split}/labels/{i + 1 :06d}.txt', 'w') as f:
            f.write(file)

In [ ]:
data: list = load_data()

In [ ]:
flatten_data_list: list[dict] = flatten_data(data)

In [ ]:
pitch_df: pd.DataFrame = get_pitch_data_df(flatten_data_list)

In [ ]:
pitch_df

In [ ]:
pitch_df_one_clip = pitch_df[pitch_df['image_id'].astype(str).str.startswith('1060')]

In [ ]:
pitch_df_one_clip

In [ ]:
def get_line_pattern(x1: float, y1: float, x2: float, y2: float) -> tuple[float, float]:
    # y = ax + b
    a = (y1 - y2) / (x1 - x2)
    b = y2 - a * x2
    return a, b

In [ ]:
def get_circle_pattern(x1: float, y1: float, x2: float, y2: float, x3: float, y3: float) -> tuple[float, float, float]:
    A = np.array([
        [x1, x2, 1],
        [x2, y2, 1],
        [x3, y3, 1]
    ])
    
    B = np.array([
        [-(x1**2 + y1**2)],
        [-(x2**2 + y2**2)],
        [-(x3**2 + y3**2)]
    ])
    
    d, e, f = np.linalg.solve(A, B).flatten()
    
    a = -d / 2
    b = -e / 2
    r = np.sqrt(pow(a, 2) + pow(b, 2) - f)
    
    return a, b, r

In [ ]:
lista = [1, 2, 3]
lista[:3]

In [ ]:
def get_func_pattern(df: pd.DataFrame) -> dict[str, list[tuple]]:
    func_patterns = {'line': [], 'circle': []}
    df_list: list[pd.DataFrame] = [image_id for _, image_id in df.groupby('image_id')]
    for df_of_image in df_list:
        image_id: int = df_of_image.iloc[0]['image_id']
        points_dict: dict[str, list] = {key: [] for key in df_of_image['line_type'].unique()}
        for i, row in df_of_image.iterrows():
            points_dict[row['line_type']].append((row['x'], row['y']))
        for line in points_dict.keys():
            if 'Circle' in line and len(points_dict[line]) >= 3:
                points: list[float, float] = points_dict[line][:3]
                flatten_points = []
                for x, y in points:
                    flatten_points.append(x * 1920)
                    flatten_points.append(y * 1080)
                func_pattern = get_circle_pattern(*flatten_points)
                func_patterns['circle'].append((image_id, *func_pattern))
            else:
                points = points_dict[line][:2]
                flatten_points = []
                for x, y in points:
                    flatten_points.append(x * 1920)
                    flatten_points.append(y * 1080)
                func_pattern = get_line_pattern(*flatten_points)
                func_patterns['line'].append((image_id, *func_pattern))
    return func_patterns

In [ ]:
lines = get_func_pattern(pitch_df_one_clip)

In [ ]:
len(lines['circle'])

In [ ]:
lines
# TODO uzupełnić o dane i narysować na obrazie te linie

In [ ]:
def draw_on_image(image: np.array, func: tuple):
    image_id = func[0]
    a = func[1]
    b = func[2]
    print(f'f(x) = {a}x + {b}')
    image = cv2.imread(image)
    print(image.shape)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    x = np.linspace(0, image.shape[1], 100)  # Zakres x, dopasowany do szerokości obrazu
    y = a * x + b
    plt.figure(figsize=(image.shape[1] / 50, image.shape[0] / 50), dpi=100)  # Dopasowanie rozmiaru
    plt.imshow(image_rgb)  # Wyświetlenie obrazu
    plt.plot(x, y, color='red', linewidth=2)
    plt.yticks(np.arange(0, image.shape[0], 20))
    plt.xticks(np.arange(0, image.shape[1], 40))
    plt.grid()
    plt.show()

In [ ]:
path = 'data/SoccerNetGS/gamestate-2024/train/SNGS-060/img1/000001.jpg'
draw_on_image(path, lines['line'][0])

In [ ]:
img = cv2.imread(path)
edited = draw_points_on_image(img, pitch_df_one_clip[pitch_df_one_clip['image_id'] == 1060000001])

In [ ]:
cv2.imshow("img with points", edited)
cv2.waitKey(0)
cv2.destroyAllWindows()